In [2]:
import numpy as np
import pandas as pd
import pickle  

In [6]:
f = open('namedic.pkl','rb')  
namedic = pickle.load(f)  
namedic

{'BAMLC0A0CMEY': 'Master_Effective_Yield',
 'BAMLC0A1CAAA': 'AAA_Option_Adjusted_Spread',
 'BAMLC0A1CAAAEY': 'AAA_Effective_Yield',
 'BAMLC0A1CAAASYTW': 'AAA_SemiAnnual_Yield_to_Worst',
 'BAMLC0A2CAA': 'AA_Option_Adjusted_Spread.csv',
 'BAMLC0A2CAAEY': 'AA_Effective_Yield',
 'BAMLC0A2CAASYTW': 'AA_SemiAnnual_Yield_to_Worst',
 'BAMLC0A3CA': 'A_Option_Adjusted_Spread',
 'BAMLC0A3CAEY': 'A_Effective_Yield',
 'BAMLC0A3CASYTW': 'A_SemiAnnual_Yield_to_Worst',
 'BAMLC0A4CBBB': 'BBB_Option_Adjusted_Spread',
 'BAMLC0A4CBBBEY': 'BBB_Effective_Yield',
 'BAMLC0A4CBBBSYTW': 'BBB_SemiAnnual_Yield_to_Worst',
 'BAMLCC0A0CMTRIV': 'Master_Total_Return_Index_Value',
 'BAMLCC0A1AAATRIV': 'AAA_Total_Return_Index_Value.csv',
 'BAMLCC0A2AATRIV': 'AA_Total_Return_Index_Value',
 'BAMLCC0A3ATRIV': 'A_Total_Return_Index_Value',
 'BAMLCC0A4BBBTRIV': 'BBB_Total_Return_Index_Value',
 'DCPF3M': '3m_AA_Financial_Commercial_Paper_Rate',
 'DCPN3M': '3m_AA_Nonfinancial_Commercial_Paper_Rate',
 'DFF': 'Effective_Federal_

In [8]:
namedic = pd.DataFrame(namedic,index = [0])
namedic.to_csv('namedic.csv')

In [3]:
data = pd.read_csv('data_fillna.csv')
data['curvature'] = 2*data['DGS2']-data['DGS3MO']-data['DGS10'] #新特征：曲率
namedic['curvature'] = 'double_2y_Minus_10y_Minus_3m_Treasury_Constant_Maturity_Rate' #添加入字典
print(data.head(2))

         DATE  T10Y2Y  T10Y3M  DGS10  DGS2  DGS3MO  BAMLC0A3CAEY  BAMLC0A3CA  \
0  1997-01-02    0.57    1.35   6.54  5.97    5.19          7.06        0.58   
1  1997-01-03    0.57    1.35   6.52  5.95    5.17          7.04        0.57   

   BAMLC0A3CASYTW  BAMLCC0A3ATRIV  ...  DCPN3M  USD3MTD156N  USEPUINDXD   DFF  \
0            7.06          212.87  ...    5.30       5.5625       40.64  5.79   
1            7.04          213.16  ...    5.29       5.5625       28.79  5.17   

   NASDAQCOM  TEDRATE   DTWEXB  WILL5000INDFC  WILLRESIND  curvature  
0    1280.70     0.51  99.2170          22.14      153.02       0.21  
1    1310.68     0.52  99.5001          22.45      152.70       0.21  

[2 rows x 48 columns]


In [23]:
#未筛选的特征 & AAA评级债券的credit spread
orig_features = ['T10Y3M','curvature','BAMLCC0A1AAATRIV','DPRIME','VIXCLS','DJIA','SP500','mktrf','smb','hml','umd','indexgx','twexb','twexm','twexo','DCPF3M','DCPN3M','DFF','NASDAQCOM','DTWEXB','TEDRATE','USD3MTD156N','USEPUINDXD','WILL5000INDFC','WILLRESIND',]
AAA_spread = data['BAMLC0A1CAAAEY'] - data['DGS10']
print(AAA_spread.head(2))

0    0.26
1    0.28
dtype: float64


In [36]:
data_features = data[orig_features]
data_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5749 entries, 0 to 5748
Data columns (total 25 columns):
T10Y3M              5749 non-null float64
curvature           5749 non-null float64
BAMLCC0A1AAATRIV    5749 non-null float64
DPRIME              5749 non-null float64
VIXCLS              5749 non-null float64
DJIA                5749 non-null float64
SP500               5749 non-null float64
mktrf               5749 non-null float64
smb                 5749 non-null float64
hml                 5749 non-null float64
umd                 5749 non-null float64
indexgx             5749 non-null float64
twexb               5749 non-null object
twexm               5749 non-null float64
twexo               5749 non-null object
DCPF3M              5749 non-null float64
DCPN3M              5749 non-null float64
DFF                 5749 non-null float64
NASDAQCOM           5749 non-null float64
DTWEXB              5749 non-null float64
TEDRATE             5749 non-null float64
USD3MTD156N  

In [37]:
#处理object数据
n = len(data_features)
for i in range(n):
    if data_features['twexb'][i] == 'N':
        data_features['twexb'][i] = data_features['twexb'][i-1]
    if data_features['twexo'][i] == 'N':
        data_features['twexo'][i] = data_features['twexo'][i-1]
data_features = pd.DataFrame(data_features,dtype=np.float)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [38]:
data_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5749 entries, 0 to 5748
Data columns (total 25 columns):
T10Y3M              5749 non-null float64
curvature           5749 non-null float64
BAMLCC0A1AAATRIV    5749 non-null float64
DPRIME              5749 non-null float64
VIXCLS              5749 non-null float64
DJIA                5749 non-null float64
SP500               5749 non-null float64
mktrf               5749 non-null float64
smb                 5749 non-null float64
hml                 5749 non-null float64
umd                 5749 non-null float64
indexgx             5749 non-null float64
twexb               5749 non-null float64
twexm               5749 non-null float64
twexo               5749 non-null float64
DCPF3M              5749 non-null float64
DCPN3M              5749 non-null float64
DFF                 5749 non-null float64
NASDAQCOM           5749 non-null float64
DTWEXB              5749 non-null float64
TEDRATE             5749 non-null float64
USD3MTD156N

In [39]:
data_features.index = data['DATE']
data_features

,T10Y3M,curvature,BAMLCC0A1AAATRIV,DPRIME,VIXCLS,DJIA,SP500,mktrf,smb,hml,...,DCPF3M,DCPN3M,DFF,NASDAQCOM,DTWEXB,TEDRATE,USD3MTD156N,USEPUINDXD,WILL5000INDFC,WILLRESIND
DATE,,,,,,,,,,,,,,,,,,,,,
1997-01-02,1.35,0.21,207.57,8.25,21.14,6442.490234,737.010010,-0.0075,0.0005,0.0013,...,5.35,5.30,5.79,1280.70,99.2170,0.51,5.56250,40.64,22.14,153.02
1997-01-03,1.35,0.21,207.86,8.25,19.13,6544.089844,748.030029,0.0139,-0.0032,-0.0078,...,5.35,5.29,5.17,1310.68,99.5001,0.52,5.56250,28.79,22.45,152.70
1997-01-06,1.37,0.23,207.66,8.25,19.89,6567.180176,747.650024,0.0002,0.0011,0.0005,...,5.34,5.32,5.26,1316.40,99.4183,0.51,5.56250,115.70,22.47,153.42
1997-01-07,1.41,0.23,207.38,8.25,19.35,6600.660156,753.229980,0.0072,-0.0002,-0.0052,...,5.33,5.32,5.20,1327.73,99.1142,0.54,5.56250,38.04,22.63,153.99
1997-01-08,1.44,0.26,207.16,8.25,20.24,6549.479980,748.409973,-0.0044,0.0050,0.0052,...,5.31,5.29,5.20,1320.35,99.2915,0.54,5.56250,63.98,22.54,154.12
1997-01-09,1.39,0.23,207.98,8.25,20.91,6625.669922,754.849976,0.0072,-0.0038,-0.0015,...,5.30,5.30,5.25,1326.20,99.2913,0.56,5.56250,6.07,22.71,155.07
1997-01-10,1.46,0.32,206.76,8.25,19.63,6703.790039,759.500000,0.0050,-0.0016,-0.0036,...,5.30,5.30,5.17,1332.02,99.3890,0.52,5.56250,45.15,22.82,155.24
1997-01-13,1.55,0.43,206.93,8.25,19.84,6709.180176,759.510010,-0.0007,0.0009,0.0027,...,5.36,5.34,5.24,1330.91,99.5418,0.55,5.59375,89.95,22.81,156.31
1997-01-14,1.34,0.24,208.04,8.25,19.27,6762.290039,768.859985,0.0111,-0.0085,0.0004,...,5.35,5.29,5.14,1346.36,99.6335,0.54,5.59375,59.83,23.07,157.39


In [41]:
AAA_spread

0       0.26
1       0.28
2       0.26
3       0.25
4       0.24
5       0.29
6       0.27
7       0.27
8       0.30
9       0.30
10      0.27
11      0.27
12      0.27
13      0.30
14      0.27
15      0.26
16      0.27
17      0.27
18      0.27
19      0.27
20      0.25
21      0.28
22      0.29
23      0.29
24      0.30
25      0.28
26      0.29
27      0.29
28      0.29
29      0.35
        ... 
5719    0.77
5720    0.79
5721    0.79
5722    0.80
5723    0.76
5724    0.73
5725    0.72
5726    0.73
5727    0.73
5728    0.73
5729    0.73
5730    0.76
5731    0.77
5732    0.78
5733    0.78
5734    0.76
5735    0.78
5736    0.84
5737    0.84
5738    0.84
5739    0.84
5740    0.86
5741    0.84
5742    0.80
5743    0.79
5744    0.77
5745    0.78
5746    0.79
5747    0.80
5748    0.79
Length: 5749, dtype: float64

In [46]:
AAA_spread = pd.array(AAA_spread,dtype=np.float)
data_features['predict'] = AAA_spread
data_features[:int(0.85*n)].to_csv('data_train.csv')
data_features[int(0.85*n):].to_csv('data_test.csv')

In [37]:
#训练集与测试集划分
X_train = data_features[:int(0.9*n)]
X_test =  data_features[int(0.9*n):]
Y_train = AAA_spread[:int(0.9*n)]
Y_test =  AAA_spread[int(0.9*n):]
print(len(X_train),len(X_test),len(Y_train),len(Y_test))

5174 575 5174 575


In [38]:
Y_train.to_csv('Y_train_nostadard.csv')
X_train.to_csv('X_train_nostadard.csv')
Y_test.to_csv('Y_test_nostadard.csv')
X_test.to_csv('X_test_nostadard.csv')

In [30]:
#数据标准化
#train_set = pd.concat([X_train,X_test],axis = 1)
#test_set = pd.concat([X_test,Y_test],axis = 1)
#print(train_set.shape,test_set.shape)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
standardized_X_train = scaler.transform(X_train)    #此时已变为np.array
standardized_X_test = scaler.transform(X_test)      #此时已变为np.array
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [32]:
# standardized_X_train = pd.DataFrame(standardized_X_train,columns=orig_features)
# Y_train = pd.DataFrame(Y_train,columns = ['predict'])
# Y_train.to_csv('Y_train.csv')
# standardized_X_train.to_csv('X_train.csv')

In [36]:
# standardized_X_test = pd.DataFrame(standardized_X_test,columns=orig_features)
# Y_test = pd.DataFrame(Y_test,columns = ['predict'])
# Y_test.to_csv('Y_test.csv')
# standardized_X_test.to_csv('X_test.csv')

In [65]:
#各阶相关系数计算函数
def corr_cal(feature,label,lag,days):
    corr_list = []
    i = 0
    while i<=len(label)-lag-days:
        df = pd.DataFrame(np.vstack((feature[i:i+days],label[i+lag:i+lag+days])))
        if df.T.corr()[0][1]<10**(-8):
            corr_list.append(0)
        else:
            corr_list.append(df.T.corr()[0][1])
        i += days
    average_corr = np.mean(corr_list)
    df = pd.DataFrame(np.vstack((feature[:len(label)-lag],label[lag:len(label)])))
    total_corr = df.T.corr()[0][1]
    return corr_list , average_corr, total_corr

In [40]:
#特征筛选（只在训练集上筛选，保持训练集与测试集的独立性）
num_of_col = X_train.shape[1]
print(num_of_col)
corr_matrix = np.zeros((31,num_of_col)) #记录15个特征，滞后0,1,...,30日的平均相关系数average_corr
total_corr_matrix = np.zeros((31,num_of_col)) #记录15个特征，滞后0,1,...,30日的总相关系数total_corr
days = 30
for i in range(num_of_col):
    for lag in range(31):
        _,corr_matrix[lag,i],total_corr_matrix[lag,i] = corr_cal(standardized_X_train[:,i],Y_train,lag,days)

25


In [41]:
index = ['LAG0','LAG1','LAG2','LAG3','LAG4','LAG5','LAG6','LAG7','LAG8','LAG9','LAG10','LAG11','LAG12','LAG13','LAG14','LAG15','LAG16','LAG17','LAG18','LAG19','LAG20','LAG21','LAG22','LAG23','LAG24','LAG25','LAG26','LAG27','LAG28','LAG29','LAG30']
average_corr_df = pd.DataFrame(corr_matrix,columns = orig_features, index = index)
total_error_df = pd.DataFrame(total_corr_matrix, columns = orig_features, index = index)

In [42]:
average_corr_df

,T10Y3M,curvature,BAMLCC0A1AAATRIV,DPRIME,VIXCLS,DJIA,SP500,mktrf,smb,hml,...,DCPF3M,DCPN3M,DFF,NASDAQCOM,DTWEXB,TEDRATE,USD3MTD156N,USEPUINDXD,WILL5000INDFC,WILLRESIND
LAG0,0.051286,0.347789,0.393924,NaN,0.272266,0.130178,0.138306,0.078880,0.067037,0.065769,...,0.231720,0.213671,0.187910,0.125903,0.320260,0.225449,0.326274,0.120665,0.135780,0.201583
LAG1,0.062457,0.324675,0.365895,NaN,0.267891,0.131432,0.139080,0.075829,0.065554,0.058398,...,0.238210,0.218872,0.175569,0.126385,0.326049,0.228058,0.334095,0.114896,0.136260,0.193564
LAG2,0.070285,0.300851,0.344309,NaN,0.269250,0.131359,0.138635,0.065170,0.070349,0.063188,...,0.242994,0.224599,0.180410,0.128586,0.326643,0.228300,0.348409,0.118152,0.135002,0.191903
LAG3,0.079350,0.281003,0.313313,NaN,0.269070,0.132759,0.140059,0.064415,0.056183,0.067590,...,0.247345,0.227719,0.185938,0.132802,0.320455,0.226891,0.353327,0.119557,0.136662,0.187671
LAG4,0.092570,0.263531,0.298610,NaN,0.267682,0.136047,0.144614,0.068274,0.059067,0.062361,...,0.251109,0.229486,0.190304,0.143306,0.311496,0.225847,0.351216,0.120050,0.143987,0.186003
LAG5,0.104880,0.250600,0.285459,NaN,0.267321,0.140410,0.149113,0.064265,0.059825,0.071036,...,0.244604,0.237773,0.183835,0.149070,0.304186,0.229165,0.353443,0.128682,0.148090,0.180964
LAG6,0.115641,0.242124,0.276595,NaN,0.258404,0.139492,0.150352,0.061854,0.064441,0.067480,...,0.249610,0.239259,0.179439,0.150018,0.293706,0.227834,0.345682,0.125246,0.148594,0.177998
LAG7,0.127139,0.236791,0.259705,NaN,0.253797,0.138663,0.150105,0.059557,0.076061,0.069298,...,0.249114,0.233265,0.183601,0.149163,0.281622,0.229519,0.341010,0.124228,0.151105,0.177893
LAG8,0.140297,0.224907,0.250592,NaN,0.242255,0.141783,0.156099,0.057552,0.075384,0.071609,...,0.258825,0.231736,0.190491,0.155245,0.277639,0.228333,0.343601,0.131575,0.158585,0.186154
LAG9,0.148527,0.216630,0.242619,NaN,0.241813,0.145222,0.158251,0.066222,0.072487,0.070927,...,0.264216,0.243985,0.196756,0.156876,0.273292,0.221238,0.348575,0.135273,0.162200,0.190110


In [43]:
total_error_df

,T10Y3M,curvature,BAMLCC0A1AAATRIV,DPRIME,VIXCLS,DJIA,SP500,mktrf,smb,hml,...,DCPF3M,DCPN3M,DFF,NASDAQCOM,DTWEXB,TEDRATE,USD3MTD156N,USEPUINDXD,WILL5000INDFC,WILLRESIND
LAG0,-0.375871,0.405965,-0.227388,0.350369,0.554331,-0.171861,-0.085061,-0.016363,-0.011120,-0.002193,...,0.398669,0.361899,0.337557,-0.052769,0.227042,0.700930,0.425130,0.061686,-0.145411,-0.201817
LAG1,-0.376204,0.407019,-0.227406,0.351316,0.554928,-0.171070,-0.084153,-0.021032,-0.012672,-0.005167,...,0.399852,0.362857,0.338593,-0.052029,0.226258,0.702967,0.426209,0.062295,-0.144791,-0.200915
LAG2,-0.376652,0.407825,-0.227350,0.352236,0.554934,-0.170107,-0.083088,-0.025885,-0.013082,-0.008844,...,0.400993,0.363836,0.339583,-0.051216,0.225390,0.704236,0.427228,0.062835,-0.144062,-0.199928
LAG3,-0.377071,0.408798,-0.227280,0.353173,0.554128,-0.169019,-0.081887,-0.029025,-0.014063,-0.010949,...,0.402175,0.364852,0.340551,-0.050307,0.224419,0.705242,0.428235,0.064098,-0.143240,-0.198831
LAG4,-0.377510,0.410053,-0.227195,0.354101,0.553063,-0.167862,-0.080601,-0.031130,-0.015557,-0.012586,...,0.403301,0.365839,0.341408,-0.049332,0.223375,0.706443,0.429246,0.066424,-0.142358,-0.197676
LAG5,-0.377769,0.411434,-0.227132,0.354993,0.551979,-0.166658,-0.079270,-0.030539,-0.015762,-0.008502,...,0.404523,0.366884,0.342140,-0.048314,0.222305,0.707773,0.430242,0.068663,-0.141448,-0.196477
LAG6,-0.378033,0.412881,-0.227007,0.355857,0.550994,-0.165453,-0.077930,-0.032963,-0.015493,-0.007403,...,0.405676,0.367846,0.342945,-0.047288,0.221240,0.709095,0.431175,0.068331,-0.140532,-0.195318
LAG7,-0.378373,0.414300,-0.226895,0.356698,0.549939,-0.164180,-0.076516,-0.035875,-0.015041,-0.005742,...,0.406787,0.368850,0.343856,-0.046206,0.220074,0.710078,0.432084,0.067862,-0.139569,-0.194149
LAG8,-0.378702,0.415650,-0.226793,0.357547,0.548353,-0.162785,-0.074996,-0.035276,-0.017051,-0.002274,...,0.407879,0.369819,0.344829,-0.045057,0.218892,0.710791,0.432964,0.068460,-0.138534,-0.192931
LAG9,-0.379065,0.416879,-0.226690,0.358379,0.546912,-0.161373,-0.073483,-0.037485,-0.017541,-0.006559,...,0.408854,0.370757,0.345746,-0.043909,0.217572,0.711172,0.433796,0.068199,-0.137498,-0.191769


In [44]:
average_corr_df.to_csv('average_corr_df.csv')

In [45]:
total_error_df.to_csv('total_corr_df.csv')

In [ ]:
X_test = np.array(dataset_test.loc[0:features]).reshape(1,-1)

features = ['T10Y3M','curvature','BAMLCC0A1AAATRIV','DPRIME','VIXCLS','DJIA','SP500','twexb','twexm','twexo','DCPF3M','DCPN3M','DFF','NASDAQCOM','DTWEXB','TEDRATE','USD3MTD156N','USEPUINDXD','WILL5000INDFC','WILLRESIND','volatility10d','ma3d','ma10d']
    training_set_scaled_x = sc_x.fit_transform(dataset_train.loc[:,features])